A quick eval script I adapted from their code:

Notes:
I trained it for 13 epochs, turns out best model by evaluation is in 5th epoch, so thats the model we will use.

How to use:

1. Change `os.chdir("/home/richard/workspaces/VML/wayfaster")`
2. Change which datapoint to evaluate by `EVAL_BATCH_IDX = 0`. It's unshuffled (in order). I made batch_size = 1, so it's just a single datapoint (sequence of images, the sequence length is 6)

Visualization outputs:

- You could check generated imgs in wandb folder, or go to wandb online.

  - Input visualization:
    - eval_depth_target is the input depth image but downsampled.
    - eval_pcloud is the input point cloud, transformed from the input depth image.
  - Output visualizatrion: eval_mu, eval_nu is the linear,angular traction coefficient.

  - The others are for Depth predictions (part of the architecture|)


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl
import os
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader
from pytorch_lightning.loggers import WandbLogger

# Custom packages
from train.dataloader import Dataset
from train.train_configs import get_cfg
from train.trainer import TrainingModule
from models.traversability_net import TravNet
from train.utils import path_to_map

In [ ]:
# Change working directory to project level
os.chdir("/home/richard/workspaces/VML/wayfaster")
print(os.getcwd())
CONFIG_FILE_PATH = "configs/temporal_model.yaml"


def parse_config():
    # Load default configs and merge with args
    config = get_cfg(CONFIG_FILE_PATH)
    return config


configs = parse_config()
print("configs:\n", configs)
pl.seed_everything(configs.SEED, workers=True)

train_dataset = Dataset(configs, configs.DATASET.TRAIN_DATA)
# valid_dataset = Dataset(configs, configs.DATASET.VALID_DATA, weights=train_dataset.weights)

train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    drop_last=True,
    pin_memory=True,
)
# valid_loader = DataLoader(
#     valid_dataset,
#     batch_size=1,
#     shuffle=False,
#     drop_last=True,
#     pin_memory=True,
# )

# Initialize model and logger (it's pl.lightining module, model.model is the actual traversability network)
model = TrainingModule(configs)
wandb_logger = WandbLogger(
    project="WayFASTER",
    log_model="all",
)

# Load a previously trained network
if configs.MODEL.LOAD_NETWORK is not None:
    print("Loading saved network from {}".format(configs.MODEL.LOAD_NETWORK))
    pretrained_dict = torch.load(configs.MODEL.LOAD_NETWORK, map_location="cpu")["state_dict"]
    model.load_state_dict(pretrained_dict)

Visualization code


In [ ]:
def visualize_results(
    logger, model, image, pcloud, trav_map, pred_depth, depth_target, depth_mask, debug, executed_path, prefix="eval"
):
    """
    Visualize the traversability network results.

    Args:
        image (torch.Tensor): Tensor containing the images.
        pcloud (torch.Tensor): Tensor containing the point clouds.
        trav_map (torch.Tensor): Tensor containing the traversability map.
        pred_depth (torch.Tensor): Tensor containing the predicted depth.
        depth_target (torch.Tensor): Tensor containing the target depth.
        depth_mask (torch.Tensor): Tensor containing the depth mask.
        debug (torch.Tensor): Tensor containing the debug information.
        executed_path (torch.Tensor): Tensor containing the executed path.
        prefix (str): Prefix for the log keys.
    """
    # Visualize the camera inputs
    logger.log_image(key=prefix + "_images", images=[image.view(-1, *image.shape[2:])])

    # Visualize the input point cloud
    pcloud = torch.mean(pcloud, dim=2, keepdim=True)
    pcloud = pcloud.view(-1, *pcloud.shape[2:])
    logger.log_image(key=prefix + "_pcloud", images=[pcloud])

    # Visualize the traversability map
    logger.log_image(key=prefix + "_mu", images=[trav_map[:, :1]])

    logger.log_image(key=prefix + "_nu", images=[trav_map[:, 1:]])

    # Visualize the depth prediction
    n_d = (model.grid_bounds["dbound"][1] - model.grid_bounds["dbound"][0]) / model.grid_bounds["dbound"][2]
    depth_pred = torch.argmax(pred_depth, dim=1, keepdim=True) / (n_d - 1)
    logger.log_image(key=prefix + "_depth_pred", images=[depth_pred])

    # Visualize the depth target
    if model.predict_depth:
        depth_target = depth_target.unsqueeze(1) / (n_d - 1)
    else:
        depth_target = depth_target.argmax(1).unsqueeze(1) / (n_d - 1)

    logger.log_image(key=prefix + "_depth_target", images=[depth_target])

    # Visualize the depth mask
    logger.log_image(key=prefix + "_depth_mask", images=[depth_mask.unsqueeze(1)])

    # Visualize the debug output
    temp = torch.sum(debug, dim=1, keepdim=True)
    temp = (temp - torch.min(temp)) / (torch.max(temp) - torch.min(temp))
    logger.log_image(key=prefix + "_debug", images=[temp])

    # Visualize the executed path
    executed_path = executed_path / (torch.amax(executed_path, (1, 2, 3), keepdim=True) + model.eps)
    logger.log_image(key=prefix + "_path", images=[executed_path])

Evaluation code:


In [ ]:
# Evaluate traversability network model
trav_model: TravNet = model.model
EVAL_BATCH_IDX = 0

eval_batch = list(train_dataset[EVAL_BATCH_IDX])
for i in range(len(eval_batch)):
    eval_batch[i] = eval_batch[i].unsqueeze(0)

with torch.no_grad():
    trav_model.eval()

    # Get data
    color_img, pcloud, inv_intrinsics, extrinsics, path, target_trav, trav_weights, depth_target, depth_mask = eval_batch  # fmt: skip
    print(f"{color_img.shape=}, {color_img.dtype=}")
    print(f"{pcloud.shape=}, {pcloud.dtype=}")

    # Forward pass
    print("Forward Pass.")
    trav_map, pred_depth, debug = trav_model(color_img, pcloud, inv_intrinsics, extrinsics, depth_target)

    # Project path to map
    print("Projecting path to map.")
    executed_path = path_to_map(
        path.unsqueeze(1),
        torch.ones_like(path[..., 0, 0]).unsqueeze(1),
        model.map_size,
        model.map_resolution,
        model.map_origin,
    )

    # Calculate traversability loss and error
    trav_loss, trav_error = model.trav_criterion(path, trav_map, target_trav, trav_weights)

    # Calculate depth classification loss
    depth_target = depth_target.view(-1, *depth_target.shape[2:])
    depth_mask = depth_mask.view(-1, *depth_mask.shape[2:])
    depth_loss = model.depth_criterion(pred_depth, depth_target, depth_mask)

    if model.train_depth:
        loss = trav_loss + model.depth_weight * depth_loss
    else:
        loss = trav_loss

    # Visualize results
    print("Logging output to Wandb")
    visualize_results(
        wandb_logger,
        model,
        color_img,
        pcloud,
        trav_map,
        pred_depth,
        depth_target,
        depth_mask,
        debug,
        executed_path,
        prefix="eval",
    )

    # Print
    print("eval_loss ", loss.item())
    print("eval_trav_loss ", trav_loss.item())
    print("eval_trav_error ", trav_error.item())
    print("eval_depth_loss ", depth_loss.item())

Trying shadow attack


In [ ]:
# Function to display the tensor
def display_tensor_images(tensor):
    """
    Display a tensor of shape (batch_size, sequence_length, 3, height, width)
    as a grid of images with rows as batch indices and columns as sequence indices.
    """
    batch_size, seq_length, channels, height, width = tensor.shape

    # Create a figure with batch_size rows and seq_length columns
    fig, axes = plt.subplots(batch_size, seq_length, figsize=(seq_length * 3, batch_size * 3))
    fig.suptitle("Tensor Visualization: Rows=Batch, Columns=Sequence", fontsize=16)

    # Ensure axes is 2D even for batch_size=1 or seq_length=1
    axes = np.array(axes).reshape(batch_size, seq_length)

    for i in range(batch_size):  # Loop over batch
        for j in range(seq_length):  # Loop over sequence
            # Extract the image for batch `i` and sequence `j`
            img = tensor[i, j].cpu().permute(1, 2, 0).numpy()  # Convert to (H, W, C) for visualization

            # Normalize image to [0, 1] if necessary
            img = (img - img.min()) / (img.max() - img.min())

            # Display the image
            axes[i, j].imshow(img)
            axes[i, j].axis("off")  # Hide axes

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Adjust for suptitle
    plt.show()

In [ ]:
from wayfaster.adversarial_attacks.shadow_attacker_optimized import ShadowAttack

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Evaluate traversability network model
trav_model: TravNet = model.model
trav_model = trav_model.to(device)
pl.seed_everything(configs.SEED, workers=True)

EVAL_BATCH_IDX = 356  # 1, 230, 1800,2700, 356

eval_batch = list(train_dataset[EVAL_BATCH_IDX])
for i in range(len(eval_batch)):
    eval_batch[i] = eval_batch[i].unsqueeze(0).to(device)

with torch.no_grad():
    trav_model.eval()

    # Get data
    color_img, pcloud, inv_intrinsics, extrinsics, path, target_trav, trav_weights, depth_target, depth_mask = eval_batch  # fmt: skip
    print(f"{color_img.shape=}, {color_img.dtype=}")
    print(f"{pcloud.shape=}, {pcloud.dtype=}")
    print(f"{path.shape=}")
    print(f"{target_trav.shape=}")
    print(f"{depth_target.shape=}")

    # Forward pass on unperturbed input
    print("Forward Pass on unperturbed input.")
    trav_map, pred_depth, debug = trav_model(color_img, pcloud, inv_intrinsics, extrinsics)

    # Visualize unperturbed input and output
    wandb_logger.log_image("unperturbed_images", images=[color_img.view(-1, *color_img.shape[2:])])
    wandb_logger.log_image("unperturbed_mu", images=[trav_map[:, :1]])
    wandb_logger.log_image("unperturbed_nu", images=[trav_map[:, 1:]])

    # unperturbed depth prediction
    n_d = (model.grid_bounds["dbound"][1] - model.grid_bounds["dbound"][0]) / model.grid_bounds["dbound"][2]
    depth_pred = torch.argmax(pred_depth, dim=1, keepdim=True) / (n_d - 1)
    wandb_logger.log_image("unperturbed_depth_pred", images=[depth_pred])

    # Perturb input data with shadow attack
    img_h, img_w = color_img.shape[-2:]
    attacker = ShadowAttack(trav_model)
    attacker.PSO_params["num_iters"] = 10
    max_l1_loss, Xp = attacker.generate_attack(model_inputs=[color_img, pcloud, inv_intrinsics, extrinsics])
    display_tensor_images(Xp)

    # Forward pass on perturbed input
    print("Forward Pass on perturbed input.")
    trav_map, pred_depth, debug = trav_model(Xp, pcloud, inv_intrinsics, extrinsics)

    # Visualize perturbed input and output
    wandb_logger.log_image("perturbed_images", images=[Xp.view(-1, *Xp.shape[2:])])
    wandb_logger.log_image("perturbed_mu", images=[trav_map[:, :1]])
    wandb_logger.log_image("perturbed_nu", images=[trav_map[:, 1:]])
    depth_pred = torch.argmax(pred_depth, dim=1, keepdim=True) / (n_d - 1)
    wandb_logger.log_image("perturbed_depth_pred", images=[depth_pred])

    # Print
    print("adv_loss ", max_l1_loss)

EVAL_BATCH_IDX = 22 # 356, 230, 120, 839,1200, 1,59, 22
